In [1]:
# Importing libraries required for connecting mysql
import pandas
import mysql.connector as sql

# Importing libraries required to get location details from lat and long
import geopy
from geopy import geocoders

# Importing the time module to measure the time elapsed in the loop
import time

In [2]:
# Connecting to MySQL

connection = sql.connect(
                        host="localhost",
                        user="root",
                        password="password",
                        database="nyc")

In [3]:
# Checking if connection is successful

connection

In [4]:
# Creating a cursor allows python to execute SQL commands

cursor = connection.cursor()

In [5]:
# Executing MySQL statement to fetch the desired columns

cursor.execute("SELECT latitude, longitude FROM nycaccidents2020 WHERE `ZIP CODE` IS NULL group by location order by count(location) desc;")

In [6]:
# cursor.fetchall() fetches all the rows of a query result as a list of tuples

lat_long = cursor.fetchall()

In [21]:
# Checking the number of records returned

len(lat_long)

5814

In [14]:
# Converting the tuples in the list to strings because geocode() takes string arguments

lat_long1 = []
for i in range(0, len(lat_long)):
    lat_long1.append(str(lat_long[i]).replace('(', '').replace(')', ''))

In [10]:
# Checking if the conversion was successful

lat_long1[0]

'40.8047, -73.91243'

In [11]:
# https://towardsdatascience.com/things-to-do-with-latitude-longitude-data-using-geopy-python-1d356ed1ae30

from geopy.geocoders import Nominatim
geocoder = Nominatim(user_agent = 'your_app_name')

In [12]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geocoder.geocode, min_delay_seconds = 1,   return_value_on_exception = None) 

# adding 1 second padding between calls

In [38]:
# Creating an empty list where the zip codes will be stored
zip = []

# Calculating the time elapsed in the loop
t0= time.perf_counter()

for i in range (0, 4000):
    location = geocode(lat_long1[i])
    zip.append(location.raw['display_name'].split(",")[-2].strip())
    
# Creating 2 for loops to avoid errors in case of network issues
for i in range (4000, len(lat_long1)):
    location = geocode(lat_long1[i])
    zip.append(location.raw['display_name'].split(",")[-2].strip())
    
t1 = time.perf_counter() - t0

In [40]:
# Time elapsed in the loop

print(t1/60, "mins")

97.73968544166668 mins


In [39]:
# Number of records fetched

len(zip)

5814

In [120]:
# Creating a dataframe that matches the latitude and logitude to the zip codes

locations = pandas.DataFrame({'lat_long':lat_long1, 'zip code':zip})

In [121]:
# Splitting the lat_long column to separate latitude and longitude columns

locations[['latitude', 'longitude']]=locations['lat_long'].str.split(',', expand=True)

In [122]:
locations

,lat_long,zip code,latitude,longitude
0,"40.8047, -73.91243",10454,40.8047,-73.91243
1,"40.651863, -73.86536",11239-2801,40.651863,-73.86536
2,"40.84506, -73.916855",10452,40.84506,-73.916855
3,"40.737785, -73.93496",11101,40.737785,-73.93496
4,"40.788795, -73.93755",10029-6408,40.788795,-73.93755
...,...,...,...,...
5809,"40.609802, -74.14108",10314,40.609802,-74.14108
5810,"40.590332, -73.9084",11697,40.590332,-73.9084
5811,"40.762917, -73.98566",10019,40.762917,-73.98566
5812,"40.68064, -73.89672",11207,40.68064,-73.89672


In [123]:
# Dropping unwanted column

locations.drop('lat_long', axis=1, inplace=True)

In [124]:
# Adding location column to match the table structure presnt in SQL

locations.insert(1, 'location', 'POINT ('+locations['longitude'].str.strip()+ ' ' +locations['latitude']+ ')')

In [125]:
locations

,zip code,location,latitude,longitude
0,10454,POINT (-73.91243 40.8047),40.8047,-73.91243
1,11239-2801,POINT (-73.86536 40.651863),40.651863,-73.86536
2,10452,POINT (-73.916855 40.84506),40.84506,-73.916855
3,11101,POINT (-73.93496 40.737785),40.737785,-73.93496
4,10029-6408,POINT (-73.93755 40.788795),40.788795,-73.93755
...,...,...,...,...
5809,10314,POINT (-74.14108 40.609802),40.609802,-74.14108
5810,11697,POINT (-73.9084 40.590332),40.590332,-73.9084
5811,10019,POINT (-73.98566 40.762917),40.762917,-73.98566
5812,11207,POINT (-73.89672 40.68064),40.68064,-73.89672


In [127]:
# Saving the dataframe as a csv file so that it can be imported into MySQL

locations.to_csv('locations_pythongenerated.csv', index=False)